In [282]:
import pandas as pd
import re

In [297]:
arlette = pd.read_csv("casas_arlette.csv")
arlette = arlette.drop_duplicates()

# Elijo unicamente los inmuebles que son casas
arlette = arlette.query("property_type=='Casa'").reset_index(drop="first")

# Elimino los acentos
arlette["address"] = arlette.address.apply(lambda x: x.lower()\
                                           .replace("á", "a")\
                                            .replace("é", "e")\
                                            .replace("í", "i")\
                                            .replace("ó", "o")\
                                            .replace("ú", "u"))

In [284]:
arlette.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          20 non-null     object 
 1   seller         20 non-null     object 
 2   property_type  20 non-null     object 
 3   address        20 non-null     object 
 4   price          19 non-null     float64
 5   bedrooms       20 non-null     float64
 6   bathrooms      19 non-null     float64
 7   built_area     19 non-null     float64
 8   land_area      19 non-null     float64
 9   description    20 non-null     object 
 10  parking        20 non-null     float64
dtypes: float64(6), object(5)
memory usage: 1.8+ KB


In [285]:
arlette = arlette.dropna()
arlette = arlette.reset_index(drop="first")

In [286]:
arlette.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          16 non-null     object 
 1   seller         16 non-null     object 
 2   property_type  16 non-null     object 
 3   address        16 non-null     object 
 4   price          16 non-null     float64
 5   bedrooms       16 non-null     float64
 6   bathrooms      16 non-null     float64
 7   built_area     16 non-null     float64
 8   land_area      16 non-null     float64
 9   description    16 non-null     object 
 10  parking        16 non-null     float64
dtypes: float64(6), object(5)
memory usage: 1.5+ KB


### Creando la columna zip_code

In [287]:
# Leo la tabla de códigos postales
tabla_CP = pd.read_html("https://codigo-postal.co/mexico/oaxaca/oaxaca-de-juarez/")
tabla_CP = tabla_CP[0]

In [288]:
# Seleccionar las columas que me sirven 
tabla_CP = tabla_CP.iloc[:, :2]
tabla_CP.drop_duplicates()

In [290]:
# Guardar en listas las colonias y sus C.P.
cp = list(tabla_CP.iloc[:, 0])
direccion = list(tabla_CP.iloc[:, 1])

In [291]:
# Crear un diccionario y Eliminar acentos de los nombres de las direcciones
cp_dict = {}
for i, j in zip(direccion, cp):
    cp_dict[i.lower()\
        .replace("á", "a")\
        .replace("é", "e")\
        .replace("í", "i")\
        .replace("ó", "o")\
        .replace("ú", "u")] = j

In [292]:
# Eliminar palabras que entorpecen la búsqueda del C.P.
arlette["address"] = arlette.address.apply(
    lambda x: x.replace("agencia", "").replace("municipal", "").strip())

In [293]:
# Creo dos listas, una con el índice de cada fila
# y la otra con el zip code que corresponde a esa fila
cp_index = []
cp_column = []
for i, d in zip(arlette.index, arlette.address):
    regex = re.compile(fr"(.{{0,20}}{d}.{{0,20}})", re.IGNORECASE)
    counter = 0
    for k, v in cp_dict.items():
        coincidencia = regex.findall(k)
        if len(coincidencia) != 0:
            cp_index.append(i)
            cp_column.append(v)
            break
        else:
            counter += 1
            if counter == 290:
                cp_index.append(i)
                cp_column.append(None)

In [294]:
# Creo la columna zip_code con los valores obtenidos de la iteración
arlette["zip_code"] = cp_column

In [295]:
arlette

,title,seller,property_type,address,price,bedrooms,bathrooms,built_area,land_area,description,parking,zip_code
0,"Residencia en Colonia Olímpica, San Felipe del...",arlette,Casa,olimpica,11000000.0,3.0,3.0,539.0,413.0,"En venta, hermosa residencia en San Felipe del...",2.0,68044.0
1,Preciosas casas nuevas en Colonia del Maestro!,arlette,Casa,del maestro,2500000.0,4.0,4.0,200.0,150.0,"En venta, dos casas completamente nuevas, en C...",2.0,68016.0
2,Hermosa casa en Lomas de la Cascada,arlette,Casa,lomas de la cascada,7500000.0,3.0,3.0,399.0,230.0,"En venta, preciosa casa, como nueva, en Fracci...",2.0,68040.0
3,Bonita casa con vistas panoramicas en San Feli...,arlette,Casa,san felipe del agua,3400000.0,2.0,2.0,155.0,310.0,"En venta, hermosa casa en San Felipe del Agua,...",2.0,68026.0
4,"Hermosa casa estilo rústico, cerca Plaza Bella!",arlette,Casa,montoya,3500000.0,4.0,4.0,220.0,600.0,"Bonita casa estilo rústico en Agencia Montoya,...",2.0,68143.0
5,"Casa en Colonia Vicente Suarez, cerca de Calza...",arlette,Casa,vicente suarez,3000000.0,7.0,3.0,338.0,278.0,"Bonita casa en Colonia Vicente Suárez, muy cer...",3.0,68156.0
6,Hermosa residencia nueva en Jardines del Valle!,arlette,Casa,jardines del valle,6800000.0,3.0,3.0,350.0,235.0,"En venta, hermosa residencia nueva, en el Frac...",2.0,68125.0
7,"Preciosa residencia en San Felipe del Agua, co...",arlette,Casa,san felipe del agua,8650000.0,4.0,3.0,190.0,736.0,"Para venta, hermosa casa, con amplio jardín ar...",2.0,68026.0
8,"Magnifica propiedad en el Centro de Oaxaca, id...",arlette,Casa,oaxaca centro,11500000.0,8.0,4.0,420.0,289.0,"En venta, excelente propiedad para oficinas, h...",2.0,68000.0
9,Hermosa residencia en Santa Lucia del Camino!,arlette,Casa,santa lucia del camino,8490000.0,5.0,5.0,522.0,475.0,"En venta, hermosa residencia en Santa Lucia de...",2.0,NaN
